In [ ]:
"""
Phishing Website Detection by Using Machine Learning Techniques - IBM Nalaiya Thiran Project
IBM Watson Studio - ML Training
Team ID: PNT2022TMID21424

Team Leader: Shri Krishnaa VN
Team Members:
Shankar Mahadevan G
Sanjeev K
Siddharthan S
Manivel Prakash
"""

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [1]:
import os
import types, ibm_boto3
import pandas as pd
from botocore.client import Config
# import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='GWKWU5L_LLB7v30OngFEtuvGaf186l-6FAg-IUPwtGCX',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'phishdetect-donotdelete-pr-ha9bj6mvggbxi8'
object_key = 'dataset.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()

,index,having_IPhaving_IP_Address,URLURL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,1,-1,1,1,1,-1,-1,-1,-1,-1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,2,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,3,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,4,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,5,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1


In [15]:
df.Result[df.Result == -1] = 0

In [16]:
x = df.drop(columns=['index', 'Result'])
y = df['Result']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 12)

In [18]:
tree = DecisionTreeClassifier(max_depth = 5)
tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5)

In [19]:
y_test_tree = tree.predict(X_test)
print(classification_report(y_test, y_test_tree))

              precision    recall  f1-score   support

           0       0.95      0.89      0.92      1001
           1       0.91      0.96      0.94      1210

    accuracy                           0.93      2211
   macro avg       0.93      0.92      0.93      2211
weighted avg       0.93      0.93      0.93      2211



In [20]:
forest = RandomForestClassifier(max_depth=5)
forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=5)

In [21]:
y_test_forest = forest.predict(X_test)
print(classification_report(y_test, y_test_forest))

              precision    recall  f1-score   support

           0       0.95      0.90      0.93      1001
           1       0.92      0.96      0.94      1210

    accuracy                           0.94      2211
   macro avg       0.94      0.93      0.93      2211
weighted avg       0.94      0.94      0.94      2211



In [22]:
xgb = XGBClassifier(learning_rate=0.4,max_depth=7)
xgb.fit(X_train, y_train)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:44:34] WARNING: /opt/conda/conda-bld/xgboost-ext_1658814095305/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.4, max_delta_step=0,
              max_depth=7, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=56,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [23]:
y_test_xgb = xgb.predict(X_test)
print(classification_report(y_test, y_test_xgb))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1001
           1       0.97      0.98      0.98      1210

    accuracy                           0.97      2211
   macro avg       0.97      0.97      0.97      2211
weighted avg       0.97      0.97      0.97      2211



In [24]:
import pickle
pickle.dump(xgb, open("XGBoostClassifier.pickle.dat", "wb"))

In [25]:
from ibm_watson_machine_learning import APIClient

In [27]:
creds = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"ZmkStuyI4oDkJ4iirb0YquWZaV0_MngDySw5q4uiA7e5"
}
client = APIClient(creds)

In [28]:
def guid(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [29]:
space_uid = guid(client, 'Phish Detect')
client.set.default_space(space_uid)
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")

In [30]:
model_details = client.repository.store_model(model=xgb,
                                              meta_props={
                                                  client.repository.ModelMetaNames.NAME:"phish_detect",
                                                  client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",
                                                  client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
                                              }
)